In [ ]:
import opentrons.execute
from opentrons import protocol_api

## General deck setup

In [ ]:
protocol = opentrons.execute.get_protocol_api("2.17")
protocol.home()

In [ ]:
# Load tip racks
tip_rack_20 = protocol.load_labware('opentrons_96_tiprack_20ul', 9)
tip_rack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', 6)

In [1]:
# Load pipette tips
p20 = protocol.load_instrument(instrument_name='p20_single_gen2', mount="right", tip_racks = [tip_rack_20])
p300 = protocol.load_instrument(instrument_name='p300_single', mount='left', tip_racks=[tip_rack_300])

In [ ]:
# Load well plates
frag_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 1)
qc_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 2)

### Add reagents to well plate

In [ ]:
# Define liquids (optional)
suspended_dna = protocol.define_liquid(
    name="Suspended DNA",
    description="Oligo fragments suspended in buffer",
    display_color="#a8329d",
)

In [ ]:
frag_wells = frag_plate.wells()[0:19].load_liquid(liquid=suspended_dna, volume=50)  # check volume

## Load modules

In [ ]:
thermocycler = protocol.load_module("thermocyclerModuleV1", 7)
thermocycler_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")

In [ ]:
temp_module = protocol.load_module("temperatureModuleV1", 3)
temp_tubes = temp_module.load_labware("opentrons_24_aluminumblock_generic_2ml_screwcap")

### Adding reagents to temperature controller

In [2]:
# Define liquids (optional)
nuclease_free_water = protocol.define_liquid(
    name="Nuclease free water",
    description="Nuclease free water",
    display_color="#0000FF",
)

pcr_mastermix = protocol.define_liquid(
    name="PCR mastermix",
    description="PCR mastermix",
    display_color="#00FF00",
)

NameError: name 'protocol' is not defined

In [ ]:
# Define reagents
water_tube = temp_tubes["A1"].load_liquid(liquid=nuclease_free_water, volume=200)  # check volume
pcr_mastermix_tube = temp_tubes["A2"].load_liquid(liquid=pcr_mastermix, volume=200)  # check volume
waste_tube = temp_tubes["A3"]

## Utility commands

In [ ]:
def print_deck_layout(protocol):
    for slot, item in protocol.deck.items():
        if item:
            print(f"Slot {slot}: {item}")
        else:
            print(f"Slot {slot}: Empty")
            
print_deck_layout(protocol)